<a href="https://colab.research.google.com/github/Abhijeet1125/adaptive-Laplacian-weight-random-forest/blob/main/ALWRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import math

In [16]:
heart_disease = pd.read_csv ( "drive/MyDrive/collabSampleData/heart-disease.csv")
x = heart_disease.drop ( "target" , axis = 1 )
y = heart_disease["target"]
np.random.seed ( 12)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 )

len(x_train)

242

just inbuilt random forest

In [17]:
np.random.seed(9)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf .get_params();

clf.fit ( x_train , y_train );

clf.score (x_test , y_test )

0.7868852459016393

-----------------------------%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%------------------------------------------------------

**CALCULATING THE COSINE SIMILARITY FOR ALL N * N CASES THEN CALCULATED TOP 5 NEIGHBOR AND FINALLY MAKING KNN MATRIX**

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(x_train, x_train )

top5_row_indices = np.argsort(cosine_sim_matrix , axis=1)[:, -6:]

n = len(x_train)
knn_matrix = np.zeros((n, n))
for i in range(n):
    for j in top5_row_indices [i]:
        knn_matrix[i][j] = cosine_sim_matrix[i][j]
        knn_matrix[j][i] = cosine_sim_matrix[j][i]

for i in range(n):
    knn_matrix[i][i] = 0 ;


**CALCULATING LAPLACIAN GRAPH AND D MATRIX**

In [19]:
D = np.zeros((n, n))

for i in range(n):
    row_sum = np.sum(knn_matrix[i])
    D[i, i] = row_sum

L = D - knn_matrix

D.shape , D , L

((242, 242),
 array([[4.99607559, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 6.99566095, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 5.98933413, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 5.99437761, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 4.99141678,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         6.99628374]]),
 array([[ 4.99607559, -0.9989127 ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.9989127 ,  6.99566095,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  5.98933413, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  5.99437761,
          0

------------------------------------------------------------------

**CALCULATING LAPLACIAN SCORE**

In [20]:
n_samples, n_features = x_train.shape

x_train_np = x_train.to_numpy()

def laplacian_score(D, L, f):
    fi = f /  np.sum(f)


    si = np.dot(fi.T, np.dot(L, fi)) / np.dot(fi.T, np.dot(D, fi))

    return si

scores = np.zeros(n_features)
for i in range(n_features ):
    sliced_matrix = x_train_np[:, i:i+1]
    si = laplacian_score(D, L, sliced_matrix)
    scores[i] = si

# Print the Laplacian score for each feature
scores

array([0.00897279, 0.29907196, 0.51008413, 0.0081261 , 0.00729152,
       0.81752487, 0.48656646, 0.00665927, 0.6202678 , 0.47142231,
       0.11901314, 0.64850664, 0.06369102])

In [21]:
fea_wei = np.array ( scores )

# please take care of this line below

fea_wei  = 1 - fea_wei


fea_wei = fea_wei / np.sum ( fea_wei)

df = pd.DataFrame ( fea_wei)
df

,0
0,0.110942
1,0.078467
2,0.054845
3,0.111037
4,0.111131
5,0.020428
6,0.057477
7,0.111201
8,0.042510
9,0.059173


**below implementation doesnot uses any thing , it just generate 100 decision tree with random selected features**

In [22]:

from sklearn.ensemble import RandomForestClassifier
num_trees = 100


predictions = []

select = x_train.shape[1]
select = math.sqrt (select)
select = int ( select )


for i in range(num_trees):
    np.random.seed ( i )
    selected_features = np.random.choice(x_train.shape[1] ,  size= select )
    subsampled_data = x_train.iloc[:, selected_features]
    subsampled_test = x_test.iloc[:, selected_features]

    tree = RandomForestClassifier(n_estimators=1, max_depth=None)
    tree.fit(subsampled_data, y_train )

    tree_predictions = tree.predict(subsampled_test)
    predictions.append(tree_predictions)


In [23]:
final_predictions = np.mean(predictions, axis=0 )


final_predictions = np.where(final_predictions  < 0.5, 0, 1)


np.mean ( final_predictions == y_test )


0.7868852459016393

**below implementation doesnot uses any thing , it generate 100 decision tree using *laplacian weight* for selecting features**

In [24]:

from sklearn.ensemble import RandomForestClassifier
num_trees = 100

predictions = []

select = x_train.shape[1]
select = math.sqrt (select)
select = int ( select )


for i in range(num_trees):
    np.random.seed ( i )
    selected_features = np.random.choice(x_train.shape[1] , p =fea_wei ,  size= select )
    subsampled_data = x_train.iloc[:, selected_features]
    subsampled_test = x_test.iloc[:, selected_features]

    tree = RandomForestClassifier(n_estimators=1, max_depth=None)
    tree.fit(subsampled_data, y_train )

    tree_predictions = tree.predict(subsampled_test)
    predictions.append(tree_predictions)


In [25]:
final_predictions = np.mean(predictions, axis=0 )


final_predictions = np.where(final_predictions  < 0.5, 0, 1)

np.mean ( final_predictions == y_test )


0.8032786885245902

**yeah! NOT MUCH BUT AROUND 2 PERCENT OF INCREASE IS FOUND**

*AS THERE IS INCREASE IN ACCURACY WE CAN SAY THAT ATLEAST LAPLACIAN SCORE IS CALCULATED CORRECTLY*

In [26]:

# just a way to generate a noise with a max of 10 percent change in value

da = np.random.choice ( 100 , 5 )


noise = np.random.rand(5 )

hk = da + 0.1 * noise * da

da , hk , noise

(array([68, 69, 52,  1, 23]),
 array([70.02383299, 69.32220047, 57.15126248,  1.00068257, 24.77052396]),
 array([0.2976225 , 0.04669572, 0.9906274 , 0.00682573, 0.76979303]))

**calculating feature importance ,**

In [27]:
from sklearn.ensemble import RandomForestClassifier
select = x_train.shape[1]
select = math.sqrt (select)
select = int ( select )

trees = 100

np.random.seed ( 75 )
x_train_inbox , x_test_outbox , y_train_inbox , y_test_outbox = train_test_split(x_train , y_train , test_size = 0.2 )

oneans=[]
noiseoob=[]
fea_imp=[]


def random_forest_with_laplacian(x_train_pass, x_test_pass, y_train_pass, y_test_pass):

  predictions = []
  for i in range(num_trees):
    np.random.seed ( i )
    selected_features = np.random.choice(x_train_pass.shape[1], p=fea_wei, size=select)

    subsampled_data = x_train_pass.iloc[:, selected_features]
    subsampled_test = x_test_pass.iloc[:, selected_features]


    tree = RandomForestClassifier(n_estimators=1, max_depth=None)
    tree.fit(subsampled_data, y_train_pass )

    tree_predictions = tree.predict(subsampled_test)
    predictions.append(tree_predictions)





  final_predictions = np.mean(predictions, axis=0 )
  final_predictions = np.where(final_predictions  < 0.5, 0, 1)
  return np.mean ( final_predictions == y_test_pass  )





oneans = random_forest_with_laplacian ( x_train_inbox , x_test_outbox , y_train_inbox , y_test_outbox  )

x_features = x_train_inbox.shape[1]

x_train_inbox

print ( x_train_inbox )

for i in range ( x_features ):


  newdata = x_train_inbox.copy()
  noise = np.random.rand(x_train_inbox.shape[0])

  value  = x_train_inbox.iloc[:,i].to_numpy()

  value = value + value * noise * 0.1


  #  please think about it

  # if i != 9 :
  #   value = value.astype ( int )

  newdata.iloc[:, i] = value

  print ( i , np.sum ( newdata.iloc[:,i] != x_train_inbox.iloc [: , i ]))

  noiseoob .append (random_forest_with_laplacian ( newdata , x_test_outbox , y_train_inbox , y_test_outbox  ) )



sum= np.sum(fea_imp)
fea_imp = abs ( oneans - noiseoob ) + 0.005
fea_imp = fea_imp / np.sum ( fea_imp)

oneans , noiseoob , fea_imp


     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
206   59    1   0       110   239    0        0      142      1      1.2   
51    66    1   0       120   302    0        0      151      0      0.4   
178   43    1   0       120   177    0        0      120      1      2.5   
276   58    1   0       146   218    0        1      105      0      2.0   
24    40    1   3       140   199    0        1      178      1      1.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
58    34    1   3       118   182    0        0      174      0      0.0   
163   38    1   2       138   175    0        1      173      0      0.0   
246   56    0   0       134   409    0        0      150      1      1.9   
247   66    1   1       160   246    0        1      120      1      0.0   
230   47    1   2       108   243    0        1      152      0      0.0   

     slope  ca  thal  
206      1   1     3  
51       1   0     2  
178      1   0    

(0.8367346938775511,
 [0.7959183673469388,
  0.8163265306122449,
  0.8367346938775511,
  0.7959183673469388,
  0.8571428571428571,
  0.8367346938775511,
  0.8367346938775511,
  0.8367346938775511,
  0.8367346938775511,
  0.8571428571428571,
  0.8163265306122449,
  0.8367346938775511,
  0.8163265306122449],
 array([0.18424292, 0.10217481, 0.02010669, 0.18424292, 0.10217481,
        0.02010669, 0.02010669, 0.02010669, 0.02010669, 0.10217481,
        0.10217481, 0.02010669, 0.10217481]))

**final check , using both laplacian and feature importances**

In [28]:
from sklearn.ensemble import RandomForestClassifier
num_trees = 100

predictions = []

select = x_train.shape[1]
select = math.sqrt (select)
select = int ( select )


for i in range(num_trees):
    np.random.seed ( i )

    new_wei = []
    for i  in range ( x_train.shape[1]):
      new_wei = (1 -i/num_trees) * fea_wei + (i / num_trees ) * fea_imp


    new_wei = new_wei / np.sum ( new_wei )
    selected_features = np.random.choice(x_train.shape[1] , p =  new_wei ,  size= select )
    subsampled_data = x_train.iloc[:, selected_features]
    subsampled_test = x_test.iloc[:, selected_features]

    tree = RandomForestClassifier(n_estimators=1, max_depth=None)
    tree.fit(subsampled_data, y_train )

    tree_predictions = tree.predict(subsampled_test)
    predictions.append(tree_predictions)



final_predictions = np.mean(predictions, axis=0 )

final_predictions = np.where(final_predictions  < 0.5, 0, 1)

np.mean ( final_predictions == y_test )


0.819672131147541